In [ ]:
import xarray as xr
import pandas as pd
import glob
import os
import numpy as np

In [ ]:
base_path = "/gws/ssde/j25b/eds_ai/frame-fm/data/"
data_path = "inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/"


In [ ]:
daily = xr.Dataset()


In [ ]:
# Descriptions of metadata are in cosmos-uk_supportinginformation_2013-2024.docx from https://data-package.ceh.ac.uk/sd/2dce161d-2fab-47bb-9fe6-38e7ed1ae18a.zip

In [ ]:
files = glob.glob(base_path + data_path + "cosmos-uk_*_hydrosoil_sh_????-????.csv")



In [ ]:
# the 12-bit mask of which QC bit files we *WANT* to allow through, set to zero to mask nothing (e.g. accept all data)
# set to 0b111111111111 to drop all data with a QC flag
qc_bitmask = 0b00000000001
#possible QC flags: M=missing, U=unchecked, I=infilled, E=estimated
#possible QC bit fields: 0=passed, 1=missing, 2=zero data, 4=too few samples
#8=low power, 16=sensor fault, 32=diagnostic fault, 64=out of range, 
#128=secondary variable, 256=midnight soil heat flux calibration, 512=spike
#1024=error code stored as value

In [126]:
dfs = []
#load each CSV into the dfs array
for file in files[0:1]:
    print(file)
    
    #check QC files exist
    qc_file=file[:-4] + "_qc_flags.csv"
    flags_file=file[:-4] + "_flags.csv"
    
    if not os.path.isfile(qc_file):
        raise FileNotFoundError("QC file " + qc_file + " not found")
    
    if not os.path.isfile(flags_file):
        raise FileNotFoundError("QC Flags file " + flags_file + " not found")
    

    #missing values should be -9999 anyway and we already turn them to NaNs
    
    df = pd.read_csv(file, delimiter=",",parse_dates=["DATE_TIME"],na_values=[-9999])
    qc_df = pd.read_csv(qc_file, delimiter=",",parse_dates=["DATE_TIME"],na_values=[-9999])
    flags_df = pd.read_csv(flags_file, delimiter=",",parse_dates=["DATE_TIME"],na_values=[-9999])

    #check the data and QC files match in shape
    assert(df.shape == qc_df.shape == flags_df.shape)

    #should we assert column names to check they're in the same order?

    #go through each column and drop data which failed to match our QC bitmask

    #first build a boolean mask
    #skip first two columns (data and site name) as they are non integer/float and not subject to QC flags
    for column in qc_df.columns[2:]:
        #applies a bitwise AND to every row in the column against our mask of acceptable values
        qc_df[column] = np.bitwise_and(qc_df[column].astype("int16"), qc_bitmask)
        
        #convert all values to booleans, by default anything non-zero becomes true, but non-zero means we want to drop it
        #so invert the output, qc_df now contains true for all entries which passed QC and false for entries which failed it
        qc_df[column] = ~qc_df[column].astype("bool")
        #qc_df.mask(qc_df[column] > 0, False, inplace=True)

    #apply the boolean mask to the data
    #anything with a true in the mask will stay, anything with a false turns into a NaN
    for column in df.columns[2:]:
        df[column] = df[column].where(qc_df[column + "_QCFLAG"])

/home/users/colinsau/cosmos-uk_balrd_hydrosoil_sh_2014-2024.csv


In [127]:
#remove timezone from the datetime as xarray's to_netcdf doesn't like it
df["DATE_TIME"] = pd.to_datetime(df.DATE_TIME).dt.tz_localize(None)
#make DATE_TIME the index instead of using a index number
dfs.append(df.set_index("DATE_TIME"))
#dfs.append(df)

#sorts all the entries by date_time as not doing can break selecting on dates
#note that there can be multiple entries with the same timestamp so the index_col entries are not unique
ds = pd.concat(dfs).sort_index().to_xarray() 

In [128]:
ds

<xarray.Dataset> Size: 3kB
Dimensions:         (DATE_TIME: 9)
Coordinates:
  * DATE_TIME       (DATE_TIME) datetime64[ns] 72B 2014-05-15T17:30:00 ... 20...
Data variables: (12/46)
    SITE_ID         (DATE_TIME) object 72B 'BALRD' 'BALRD' ... 'BALRD' 'BALRD'
    LWIN            (DATE_TIME) float64 72B 339.4 329.4 316.6 ... 336.2 324.0
    LWOUT           (DATE_TIME) float64 72B 398.6 396.3 387.7 ... 372.5 369.3
    SWIN            (DATE_TIME) float64 72B 198.3 178.3 83.9 ... -1.965 -2.834
    SWOUT           (DATE_TIME) float64 72B 32.77 30.27 13.6 ... 0.821 0.804
    RN              (DATE_TIME) float64 72B 106.3 81.13 -0.8 ... -39.09 -48.94
    ...              ...
    TDT10_VWC       (DATE_TIME) float64 72B nan nan nan nan nan nan nan nan nan
    STP_TSOIL2      (DATE_TIME) float64 72B 17.39 16.96 16.21 ... 13.18 12.89
    STP_TSOIL5      (DATE_TIME) float64 72B 17.44 17.07 16.62 ... 14.07 13.75
    STP_TSOIL10     (DATE_TIME) float64 72B 16.73 16.54 16.32 ... 14.81 14.51
    STP_TSOIL20     (DATE_TIME) float64 72B 14.92 14.92 14.89 ... 14.56 14.45
    STP_TSOIL50     (DATE_TIME) float64 72B 11.49 11.48 11.47 ... 11.48 11.49

In [129]:
# free up some memory, by getting rid of the Pandas version of the data
# frees about 2GB
dfs = None

In [ ]:
ds.sel(DATE_TIME=slice("2014-05-15 17:30:00","2014-05-15 20:00:00"))

In [131]:
ds.to_netcdf("/gws/ssde/j25b/eds_ai/frame-fm/users/colinsau/cosmos-uk-hydrosoil.nc")